<a href="https://colab.research.google.com/github/rescobarcorrea/GEE_SST_time_series_extractor/blob/main/GEE_SST_time_series_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Satellite sensed SST at specific locations on the east coast of Australia**




## 0. Install stuff

In [ ]:
!pip install -U -q earthengine-api
!pip install -U -q folium
!pip install -U -q git+https://github.com/mccarthyryanc/folium_gee.git

#!pip install geopandas
#!pip install descartes 
!pip install geemap

     |████████████████████████████████| 153kB 7.5MB/s 
     |████████████████████████████████| 102kB 5.6MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.11.0 which is incompatible.
     |████████████████████████████████| 378kB 8.0MB/s 
     |████████████████████████████████| 4.7MB 17.8MB/s 
     |████████████████████████████████| 501kB 45.8MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 5.1MB 44.1MB/s 
     |████████████████████████████████| 1.1MB 42.1MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 225kB 47.9MB/s 
     |████████████████████████████████| 122kB 43.3MB/s 
  Created wheel for geemap: filename=geemap-0.7.1-py2.py3-none-any.whl size=361899 sha256=9e42d7cb55f98bc003219b748b26469395fd53d0012d8baf15602e8a98381365
  Stored in directory: /root/.cache/pip/wheels/cb/aa/a4/f77e2fa45d2bbc62c3a9730abca590a98166c2e3ebb7b5d8d6
  Created wheel fo

## 1. Authenticate

In [ ]:
# Click here to authenticate
!earthengine authenticate 

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=vaL8ySueTsACXlCjvi8UpUs_DxERCL8cHP7PPFHRYXs&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/0wEqAASp2bj9qtRUPvwx0BtDjypugUUnERFbYkAdQLIjpNXg8jk8XUA

Successfully saved authorization token.


## 2. Start it up 

In [ ]:
import ee
import folium
import foliumgee
import pandas as pd
#import geopandas as gpd
import subprocess
import os
import numpy as np
from google.colab import files
#from datetime import datetime
from datetime import datetime
from datetime import timedelta
from datetime import date
import time
from dateutil.parser import parse
try:
        import geemap
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
        import google.colab
        import geemap.eefolium as emap
except:
        import geemap as emap


from google.colab import drive
drive.mount('/content/drive')  

try:
  ee.Initialize()
  print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
  print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'    

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Google Earth Engine has initialized successfully!


## 3. Get inputs

In [ ]:
#3.1 creates a freature with the coordinates of the points
points=ee.Geometry.MultiPoint([[153.148101,-30.304086],[153.143744,-30.267377],[153.293476,-29.8153],[153.143176,-30.295505],[153.30166,-29.771711],[153.109003,-30.374972],[153.2777,-30.108432]])
ids=pd.Series(['Breakwater Coffs','Charlesworth Bay','Diggers Camp','Little Muttonbird','Minnie Water,Sawtell','Woolgoolga'])
point=ee.Geometry.Point([153.148101,-30.304086])
#3.2 Get list of dates
dates=("2016-08-15","2018-09-25","2018-10-02","2018-10-03","2018-10-03","2019-07-24","2019-09-28","2019-11-20","2019-12-04","2019-12-10")

resolution=1

In [ ]:
print(dates[0])

2016-08-15


## 4. Get SST per sampling dates

In [ ]:
temperatures=pd.DataFrame()
for single_date in dates:
  sd= (parse(single_date)-timedelta(days=1)).strftime('%Y-%m-%d')
  #ed= (parse(end_date)+timedelta(days=4)).strftime('%Y-%m-%d')
  print(single_date)
  OISST = ee.ImageCollection('NOAA/CDR/OISST/V2').filterDate(sd,single_date)
  sst=ee.Image(OISST.select('sst').first())
  dist_stats = sst.sample(points,10)
  dist_stats = dist_stats.select(['sst'],[single_date],retainGeometry=False).getInfo()
  #print(dist_stats)
  dtstats_df = pd.DataFrame()
  for dist in dist_stats['features']:
        df = pd.DataFrame([dist['properties']], columns=dist['properties'].keys())
        #print('.')
        dtstats_df = pd.concat([dtstats_df, df], axis=0)
    #print('-')   
  temperatures = pd.concat([temperatures, dtstats_df], axis=1)
  temperatures/100

2016-08-15
2018-09-25
2018-10-02
2018-10-03
2018-10-03
2019-07-24
2019-09-28
2019-11-20
2019-12-04
2019-12-10


### Temperatures per year

In [ ]:
start_date= ('2019-01-01')# minimum 2007-01-01
end_date= ('2020-01-01')# minimum 2007-01-01

daterange = pd.date_range(start_date, end_date)
tempyear=pd.DataFrame()
for single_date in daterange:
  print(single_date)
  OISST = ee.ImageCollection('NOAA/CDR/OISST/V2').filterDate(single_date)
  sst=ee.Image(OISST.select('sst').first())
  dist_stats = sst.sample(points,10)
  dist_stats = dist_stats.select(['sst'],[single_date.strftime('%Y-%m-%d')],retainGeometry=False).getInfo()
  #print(dist_stats)
  dtstats_df = pd.DataFrame()
  for dist in dist_stats['features']:
        df = pd.DataFrame([dist['properties']], columns=dist['properties'].keys())
        #print('.')
        dtstats_df = pd.concat([dtstats_df, df], axis=0)
    #print('-')   
  tempyear = pd.concat([tempyear, dtstats_df], axis=1)
#tempyear.to_csv('tempyear.csv')
tempyear/100
tempyear


2019-01-01 00:00:00
2019-01-02 00:00:00
2019-01-03 00:00:00
2019-01-04 00:00:00
2019-01-05 00:00:00
2019-01-06 00:00:00
2019-01-07 00:00:00
2019-01-08 00:00:00
2019-01-09 00:00:00
2019-01-10 00:00:00
2019-01-11 00:00:00
2019-01-12 00:00:00
2019-01-13 00:00:00
2019-01-14 00:00:00
2019-01-15 00:00:00
2019-01-16 00:00:00
2019-01-17 00:00:00
2019-01-18 00:00:00
2019-01-19 00:00:00
2019-01-20 00:00:00
2019-01-21 00:00:00
2019-01-22 00:00:00
2019-01-23 00:00:00
2019-01-24 00:00:00
2019-01-25 00:00:00
2019-01-26 00:00:00
2019-01-27 00:00:00
2019-01-28 00:00:00
2019-01-29 00:00:00
2019-01-30 00:00:00
2019-01-31 00:00:00
2019-02-01 00:00:00
2019-02-02 00:00:00
2019-02-03 00:00:00
2019-02-04 00:00:00
2019-02-05 00:00:00
2019-02-06 00:00:00
2019-02-07 00:00:00
2019-02-08 00:00:00
2019-02-09 00:00:00
2019-02-10 00:00:00
2019-02-11 00:00:00
2019-02-12 00:00:00
2019-02-13 00:00:00
2019-02-14 00:00:00
2019-02-15 00:00:00
2019-02-16 00:00:00
2019-02-17 00:00:00
2019-02-18 00:00:00
2019-02-19 00:00:00


,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,2019-01-11,2019-01-12,2019-01-13,2019-01-14,2019-01-15,2019-01-16,2019-01-17,2019-01-18,2019-01-19,2019-01-20,2019-01-21,2019-01-22,2019-01-23,2019-01-24,2019-01-25,2019-01-26,2019-01-27,2019-01-28,2019-01-29,2019-01-30,2019-01-31,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-02-05,2019-02-06,2019-02-07,2019-02-08,2019-02-09,...,2019-11-23,2019-11-24,2019-11-25,2019-11-26,2019-11-27,2019-11-28,2019-11-29,2019-11-30,2019-12-01,2019-12-02,2019-12-03,2019-12-04,2019-12-05,2019-12-06,2019-12-07,2019-12-08,2019-12-09,2019-12-10,2019-12-11,2019-12-12,2019-12-13,2019-12-14,2019-12-15,2019-12-16,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-21,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31,2020-01-01
0,2486,2417,2386,2404,2398,2478,2522,2554,2549,2509,2438,2447,2445,2487,2449,2540,2537,2446,2449,2498,2522,2438,2335,2409,2344,2548,2578,2592,2599,2603,2619,2505,2584,2642,2723,2707,2681,2656,2642,2634,...,2216,2159,2104,2103,2099,2223,2249,2248,2197,2213,2235,2297,2347,2344,2344,2389,2429,2441,2424,2401,2343,2370,2396,2391,2342,2314,2318,2346,2399,2414,2390,2371,2380,2416,2453,2464,2486,2453,2444,2451
0,2484,2417,2386,2318,2398,2478,2522,2554,2478,2428,2417,2447,2445,2487,2495,2540,2502,2446,2449,2498,2522,2438,2335,2409,2486,2548,2578,2592,2599,2603,2619,2662,2660,2683,2723,2707,2681,2656,2642,2634,...,2216,2159,2104,2103,2099,2223,2249,2248,2197,2213,2235,2297,2347,2359,2344,2389,2429,2441,2424,2404,2343,2370,2396,2391,2342,2314,2318,2346,2399,2414,2390,2371,2380,2416,2453,2464,2486,2453,2444,2395
0,2486,2484,2472,2318,2460,2521,2565,2593,2478,2428,2417,2459,2480,2432,2495,2563,2502,2501,2502,2520,2511,2405,2292,2343,2486,2315,2267,2280,2321,2346,2369,2662,2660,2683,2702,2697,2685,2668,2661,2646,...,2191,2174,2064,2041,2046,2169,2205,2217,2221,2270,2291,2333,2367,2359,2306,2333,2376,2409,2412,2404,2377,2387,2388,2385,2323,2280,2298,2330,2369,2393,2361,2379,2410,2427,2431,2385,2429,2457,2463,2395
0,2486,2489,2464,2399,2463,2526,2560,2588,2537,2518,2502,2459,2410,2432,2510,2520,2559,2508,2506,2534,2535,2433,2325,2390,2449,2512,2267,2563,2577,2580,2591,2505,2677,2694,2702,2717,2695,2668,2653,2640,...,2253,2221,2129,2041,2101,2244,2286,2291,2269,2290,2301,2335,2368,2363,2338,2383,2418,2435,2422,2402,2344,2367,2388,2387,2348,2330,2343,2377,2422,2434,2394,2394,2420,2447,2479,2477,2507,2490,2463,2463
0,2486,2484,2472,2404,2460,2521,2560,2588,2549,2509,2438,2459,2410,2432,2449,2520,2537,2501,2502,2520,2511,2405,2292,2343,2344,2315,2267,2280,2321,2346,2369,2505,2584,2642,2702,2697,2685,2668,2661,2646,...,2191,2174,2064,2041,2046,2169,2205,2217,2221,2270,2291,2333,2367,2344,2306,2333,2376,2409,2412,2401,2377,2387,2388,2385,2323,2280,2298,2330,2369,2393,2361,2379,2410,2427,2431,2385,2429,2457,2463,2451
0,2409,2484,2472,2404,2460,2521,2560,2588,2549,2509,2438,2459,2410,2432,2449,2520,2537,2501,2502,2520,2511,2405,2292,2343,2344,2315,2267,2280,2321,2346,2369,2505,2584,2642,2702,2697,2685,2668,2661,2646,...,2191,2174,2064,2041,2046,2169,2205,2217,2221,2270,2291,2333,2367,2344,2306,2333,2376,2409,2412,2401,2377,2387,2388,2385,2323,2280,2298,2330,2369,2393,2361,2379,2410,2427,2431,2385,2429,2457,2463,2451
0,2409,2484,2472,2404,2460,2521,2560,2588,2549,2509,2438,2523,2410,2504,2449,2520,2537,2501,2502,2520,2511,2405,2292,2343,2344,2315,2530,2280,2321,2346,2369,2668,2584,2642,2732,2697,2685,2666,2661,2646,...,2191,2174,2064,2109,2046,2169,2205,2217,2221,2270,2291,2333,2367,2344,2306,2333,2376,2409,2412,2401,2377,2387,2391,2385,2323,2280,2298,2330,2369,2393,2361,2379,2410,2427,2431,2385,2429,2457,2495,2451


## 5. Visualization map (if you want)

In [ ]:
# Introduce a date of an image you want to visualize
datevis="2016-08-15"
# New map vizualization with geemap
OISST = ee.ImageCollection('HYCOM/sea_temp_salinity').filterDate(datevis)
sst=ee.Image(OISST.select('water_temp_0').first())
center_aus=[-30.304080,153.148101]
Map1 = emap.Map(center=center_aus, zoom_start=9)

palette= [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]
  #Add SST
sstvis= {
  'min': 1500,
  'max': 3500,
  'palette': palette}
Map1.addLayer(sst, sstvis, 'SST')
#Add Points 
Map1.addLayer(points, {'color': 'ff0000'}, 'Points')
# Show map
Map1.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map1

## 6. Save info

In [ ]:
temperatures.to_csv('temperatures.csv')
files.download('temperatures.csv')

tempyear.to_csv('tempyear.csv')
files.download('tempyear.csv')